## Set Run, Sample, and Cosmo Parameters 
<br>
  
#### Syntax
`RunPars, CosmoPars, HeliumPars = SetParams(InputPars)` <br>
`RunPars = SetRunParameters(erosion_type, total_time, erosion_rate, scenarios, c, MAT, T_AMP)` <br>
`CosmoPars, HeliumPars = SetCRNProdParameters()` <br>
`CosmoPars, HeliumPars = SetCalcParameters(RunPars,CosmoPars, HeliumPars)` <br>
#### Input 
`SetParams()` <br>
`total_time`: total run time (years) <br>
`erosion_rate`: imposed erosion rate (cm yr-1) <br>
`scenarios`: scenarios (no.) <br>
`c` : pareto shape value <br>
`MAT` : mean annual temperature (C) <br>
`T_AMP` : seasonal temperature amplitude (C) <br>
#### Output
`RunPars` : dictionary of size 9 with the information relevant to the run. <br>
`CosmoPars` : dictionary of size 89 with CRN production parameters <br>
`HeliumPars` : dictionary of size 10 with Helium diffusion parameters <br>
#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [14]:
def SetParams(InputPars):
    total_time = InputPars['Total Time']
    erosion_rate = InputPars['Erosion Rate']
    scenarios = InputPars['Scenarios']
    c = InputPars['c']
    MAT = InputPars['MAT']
    T_AMP = InputPars['T_AMP']
    
    RunPars = SetRunParameters(total_time, erosion_rate, scenarios, c, MAT, T_AMP)
    CosmoPars, HeliumPars = SetCRNProdParameters()
    CosmoPars,HeliumPars = SetCalcParameters(RunPars,CosmoPars, HeliumPars)
    return RunPars, CosmoPars, HeliumPars

In [3]:
def SetRunParameters(total_time, erosion_rate, scenarios, c, MAT, T_AMP):
    RunParameters = {'total_time' : total_time,
                 'erosion_rate' : erosion_rate, # imposed erosion rate
                 'scenarios' : scenarios, # number of scenarios
                     'c' : c, # Pareto shape parameter
                    'MAT' : MAT, # mean annual temperature (C)
                    'T_AMP' : T_AMP, # Temperature amplitude (C)
                    'sample_no': 30, # nr
                'max_depths' : [350,5000], # cm [0] is for Helium and [1] is for CRNs (C14/10Be)
                    'dz' : 1.0} # grid spacing with depth
             
    return RunParameters

In [46]:
def SetCRNProdParameters():
    CosmoParameters = {'SLHL_C14' : 12.76, # at g a-1
                    'SLHL_Be10' : 4.01, # at g a-1
                    'SLHL_He3' : 128, # at g a-1
                    'scaling_factor' : 1.0, 
                      'rock_rho': 2.7, # g cm-3
                      'CN_lambda': 160, # g cm2,
                       'dt' : 1,
                      'Be10 Decay': 4.998e-7,
                       'B_Be': 0.026, # Granger an Smith 2000
                      'L3 Be' : 4360, # g cm-2 Granger and Smith 2000
                      'C14 thalf' : 5730,
                      'C14 f_tot' : 1.72*10**-2, # Lupker et al 2015
                      'C14 A1' : 170.6, # Granger and Smith (2000)
                      'C14 A2' : 36.75, # Granger and Smith 2000
                      'L1 C14': 788.6, # g cm-2 Granger and Smith
                      'L2 C14' : 2688} # g cm2 Granger and Smith   
    
    HeliumParameters = {'nx': 100, # grain nodes
                       'r' : 0.1, # grain size}
                        'alpha' : 1.5e-2, # bedrock diffusivity (cm2 s)
                       'qtz_rho': 2.65, # g cm-3
                        'siy' : 86400*365,
                        'dt' : 12, # hours
                       'EDT_Ea' : 112*1e3,
                        'He_Ea' : 84500,
                       'R' : 8.3144621,
                       'a' : 0.043,
                       'lnD0aa' : 11.1,
                       'thermsigma' : 0.5,
                       'Normal T Fxn' : True} 
                            
    return CosmoParameters, HeliumParameters

In [59]:
def SetCalcParameters(RunPars,CosmoPars,HeliumPars):
    max_depths = RunPars['max_depths']
    dz = RunPars['dz']
    dt = HeliumPars['dt']
    alpha = HeliumPars['alpha']
    siy = HeliumPars['siy']
    
    CosmoPars['mu'] = CosmoPars['rock_rho'] / CosmoPars['CN_lambda'] # calculate mu
    CosmoPars['CRNz'] = np.arange(0,max_depths[1],dz)  # vector of depths for the CRNs
    CosmoPars['C14 decay'] = np.log(2) / CosmoPars['C14 thalf'] # calculate decay constant C14
    CosmoPars['CRNnz'] = int(max_depths[1] / dz)
    
    qtz_rho = HeliumPars['qtz_rho']
    r = HeliumPars['r']
    nx = HeliumPars['nx']
    dx = (r/nx) # grid spacing
    x = np.arange(dx/2,r,dx) # node locations
    x_up = x + (dx/2) # outer edge of shell
    x_low = x - (dx/2) # inner edge of shell
    x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))] #shell volumes
    shell_mass = [(x_vol[i] * qtz_rho) for i in range(len(x_vol))] # shell mass
    Hez = np.arange(0,max_depths[0]+1,dz)
    sigma = HeliumPars['thermsigma']
    alpha_cmhr = 0.0152*60*60 # diffusivity cm2 / hour   
    thermnz = max_depths[0]+1 # number of thermal nodes
    thermdz = (max_depths[0]+1) / (thermnz - 1) # node spacing
    thermnt = 8760 # number of time nodes (equals 1 year)
    thermdt = sigma * thermdz**2 / alpha_cmhr # dt in seconds
    
    HeliumPars['nz'] = int((max_depths[0] + 1) / dz)
    HeliumPars['T0'] = np.array([RunPars['MAT']] * thermnz)
    HeliumPars['dx'] = dx
    HeliumPars['x'] = x 
    HeliumPars['x_up'] = x_up
    HeliumPars['x_low'] = x_low
    HeliumPars['x_vol'] = x_vol
    HeliumPars['shell_mass'] = shell_mass
    HeliumPars['Hez'] = Hez
    HeliumPars['thermnz'] = thermnz 
    HeliumPars['thermdt'] = thermdt
    HeliumPars['thermdz'] = thermdz
    HeliumPars['thermnt'] = thermnt
    HeliumPars['alpha_cmhr'] = alpha_cmhr

    return CosmoPars, HeliumPars